In [46]:
import pandas as pd
import numpy as np


In [47]:
dataset = pd.read_csv('PlayTennis.csv', names = ['outlook','temperature','humidity','wind','class'])

In [60]:
def entropy(target_col):
    elements, counts = np.unique(target_col, return_counts =True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
#     entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [61]:
def infoGain(data,split_target, target_name='class'):
    total_entropy = entropy(data[target_name])
    val, counts = np.unique(data[split_target], return_counts = True)
    weighted_entropy = np.sum([(counts[i]/np.sum(counts))*entropy(data.where(data[split_target]==val[i]).dropna()[target_name])for i in range(len(val))])
    informationGain = total_entropy - weighted_entropy
    return informationGain

In [62]:
def ID3(data, originaldata, features, target_name = 'class', parent_node_class = None):
    if len(np.unique(data[target_name]))<=1:
        return np.unique(data[target_name])[0]
    elif len(data)==0:
        return np.unique(originaldata[target_name])[np.argmax(np.unique(originaldata[target_name],return_counts = True)[1])]
    elif len(features) == 0:
        return parent_node_class

    else:
        parent_node_class = np.unique(data[target_name])[np.argmax(np.unique(data[target_name],return_counts=True)[1])]
    items = [infoGain(data,feature,target_name) for feature in features]
    best_feature_index = np.argmax(items)
    best_feature = features[best_feature_index]
    tree = {best_feature : {}}
    features = [i for i in features if i != best_feature]
    
    for value in np.unique(data[best_feature]):
        value = value
        subData = data.where(data[best_feature] == value).dropna()
        subTree = ID3(subData, dataset, features, target_name,parent_node_class)
        tree[best_feature][value] = subTree
    return tree
    

In [63]:
tree = ID3(dataset, dataset, dataset.columns[:-1])
print(tree)

{'outlook': {'Overcast': 'Yes', 'Rain': {'wind': {'Strong': 'No', 'Weak': 'Yes'}}, 'Sunny': {'humidity': {'High': 'No', 'Normal': 'Yes'}}}}
